For the second half of FY23 (Apr - Sep), what is our gross charge split for peds (\<18 yrs old on date of admit) vs other patients?

In [22]:
/*
    Checked the BIRTH_DT column for NULL values and found none. Thus the INNER JOIN vs a LEFT JOIN.
    The CASE statements are used to determine if the patient is a pediatric patient or not. If the patient is 18 years or older, they are considered an adult.
    Because the question is asking about FY dates, used the PostingDate column to filter the data.
*/

SELECT
    CASE
        WHEN DATEDIFF(year, P.BIRTH_DT, AN.ACCT_ADMIT_DT) >= 18 THEN 'Other Patients'
        ELSE 'Peds (<18 yrs old)'
    END AS PatientType,
    SUM(T.LineCharges) AS TotalCharges
FROM
    dbo.Transactions AS T
    INNER JOIN dbo.Account_Number AS AN 
        ON T.ACCOUNT_NUMBER = AN.ACCOUNT_NUMBER
    INNER JOIN dbo.Patient AS P 
        ON AN.MRN = P.MRN
WHERE
    T.PostingDate >= '2023-04-01' -- Start date of the second half of FY23 (April 1, 2023)
    AND T.PostingDate < '2023-10-01' -- End date of the second half of FY23 (September 30, 2023)
GROUP BY
    CASE
        WHEN DATEDIFF(year, P.BIRTH_DT, AN.ACCT_ADMIT_DT) >= 18 THEN 'Other Patients'
        ELSE 'Peds (<18 yrs old)'
    END;

(2 rows affected)

Total execution time: 00:00:00.071

PatientType,TotalCharges
Peds (<18 yrs old),7398025.83
Other Patients,22954497.97


What revenue code incurred the most gross charges for patients over 65 years old and insured by a Medicare product in FY23 (Oct - Sep)?

In [25]:
/*
	The assumption in this query is that 'Medicare' in the AN.PRIM_FIN_CLASS_NM  is the correct financial class grouping for Medicare products.
	'Medicare Managed' or Advantage Plans, is a potential canidate to include in the query but further clarification would be needed.

	If I include 'Medicare' and 'Medicare Managed' in the query, the rev code that included the most gross charges is rev code 0301 - Laboratory - Clinical Chemistry. Total charges of 426462.15.
	
	Otherwise with just 'Medicare' it's Revcode 202 - Intensive Care Unit. Total charges of 354744.00.
*/

SELECT TOP (1)
	T.Revcode,
	RC.REVENUE_CODE_NAME AS RevenueCodeName,
	SUM(T.LineCharges) AS TotalCharges
FROM 
	dbo.Transactions AS T
	INNER JOIN dbo.Account_Number AS AN
		ON T.ACCOUNT_NUMBER = AN.ACCOUNT_NUMBER
	INNER JOIN dbo.Patient AS P
		ON AN.MRN = P.MRN
	INNER JOIN dbo.Rev_Codes AS RC
		ON T.Revcode = RC.REVENUE_CODE
WHERE 
	P.BIRTH_DT <= DATEADD(year, -65, GETDATE()) --Patients over 65
	AND AN.PRIM_FIN_CLASS_NM = ('Medicare') --Insured by Medicare
	AND T.PostingDate >= '2022-10-01' --FY23 start date
	AND T.PostingDate < '2023-10-01' --FY23 end date
GROUP BY 
	T.Revcode,
	RC.REVENUE_CODE_NAME
ORDER BY 
	TotalCharges DESC;


(1 row affected)

Total execution time: 00:00:00.164

Revcode,RevenueCodeName,TotalCharges
0202,INTENSIVE CARE-MEDICAL,354744.00


Which provider generated the most gross charges for female patients in the first half of FY23 (Oct - Mar)?

In [16]:
/*
  Need to create three joins to get to the get the Provider Description column.
  Because the column VST_PROV_NPI in the Account_Number table is blank for about 5,722 rows, those rows will be discarded when joining on the Provider table with an INNER JOIN.
  This is not an issue since the the question is asking specifically about the top provider with the most charges. 
*/

SELECT TOP (1)
    P.[Provider Description] AS ProviderDescription,
    SUM(T.LineCharges) AS TotalCharges
FROM
    dbo.Transactions AS T
    INNER JOIN dbo.Account_Number AS AN 
		  ON T.ACCOUNT_NUMBER = AN.ACCOUNT_NUMBER
    INNER JOIN dbo.Patient AS PT 
		  ON AN.MRN = PT.MRN
    INNER JOIN dbo.Provider AS P 
		  ON AN.VST_PROV_NPI = P.NPI
WHERE
	T.PostingDate >= '2022-10-01' -- Start date of the first half of FY23 (October 1, 2022)
    AND T.PostingDate < '2023-04-01' -- End date of the first half of FY23 (April 1, 2023)
    AND PT.SEX = 'F' --Female patients
GROUP BY
    P.[Provider Description]
ORDER BY
	TotalCharges DESC;

(1 row affected)

Total execution time: 00:00:00.130

ProviderDescription,TotalCharges
PROV_0951,187397.97


How many unique patients were admitted to the hospital and incurred a room and board charge (defined via rev code) in FY23?

In [20]:
/*
	Decided to use the Account_Number table to count the total number of unique patients based on the fact it holds the ACCT_ADMIT_DT and the question is specifically asking about admited patients.
	Need to join on the Transactions table and then the Rev_Codes table to get the REVENUE_CODE_NAME to filter on the Room & Board revenue code.
	It is possible for patients to have multiple accounts, so I used the DISTINCT keyword on MRN to only count each patient once.
	There are other ways to filter on the room and board charge. I could have used the LIKE keyword on the [Revcode Desc] in the transactions table. Or I could have used the IN statement
	on the Revcode after identifying the correct rev codes. I chose to use the LIKE keyword on the REVENUE_CODE_NAME to show how the tables relate to each other.
*/

SELECT 
	COUNT(DISTINCT AN.MRN) AS UniquePatients
FROM dbo.Account_Number AS AN
	INNER JOIN dbo.Transactions AS T 
		ON AN.ACCOUNT_NUMBER = T.ACCOUNT_NUMBER
	INNER JOIN dbo.Rev_Codes AS RC 
		ON T.Revcode = RC.REVENUE_CODE
WHERE T.PostingDate >= '2022-10-01' -- Start date of FY23 (October 1, 2022)
    AND T.PostingDate < '2023-10-01' -- End date of FY23 (October 1, 2023)
    AND RC.REVENUE_CODE_NAME LIKE 'ROOM & BOARD%';

(1 row affected)

Total execution time: 00:00:00.074

UniquePatients
375


What month had the most inpatient account volume where total charges per account were \>$25,000?

In [14]:
/*
  	A few impotant assumptions in this query. 
  	
	First, I assumed ACCT_DICSH_DT as the month column since it's not explicity stated in the question by which date field to utilize. 
	Depending on which date field is used, the results will be different.
  
  	Second, I assumed that the TOT_CHGS column for the WHERE statement. The question is focusing on inpatient account volumes and not related to transactions, posting dates, CPT and rev codes, etc.
	I believe it was safe to use the TOT_CHGS column as a filter because this total matches the total in the Transactions table.
  	
	Third, I assumed that the VISIT_BASECLS_NM column is the best column to use for filtering inpatient accounts based on the limited number of options in the column making grouping easier.	
*/

SELECT TOP (1)
	DATENAME(MONTH, ACCT_DISCH_DT) AS DischargeMonth,
	COUNT(DISTINCT ACCOUNT_NUMBER) AS AccountVolume
FROM dbo.Account_Number AS AN
WHERE 
	TOT_CHGS > 25000
	AND VISIT_BASECLS_NM = 'Inpatient'
GROUP BY
	DATENAME(MONTH, ACCT_DISCH_DT)
ORDER BY AccountVolume DESC;
	

(1 row affected)

Total execution time: 00:00:00.028

DischargeMonth,AccountVolume
March,33


Bonus 1 Create a script that will be accessed by Tableau that includes the following fields: 

|  |
| --- |
| Account Number |
| Rev Code |
| Rev Code Description |
| Age at Admission |
| Adm Date |
| Disch Date |
| CPT Code |
| Visit Provider |
| Visit Provider Description |
| Total Charges |

In [10]:
/*
    Create a view called vTableau that will be used to create the Tableau data source. 
    Used a LEFT JOIN on Provider table to ensure that all records from the Transactions and Account_Number tables are included in the view.
    Alias all columns apporpriately to match the requirements for the question.
*/

CREATE VIEW dbo.vTableau
AS
SELECT
    AN.ACCOUNT_NUMBER AS [Account Number],
    T.Revcode AS [Rev Code],
    T.[RevCode Desc] AS [Revenue Code Description],
	DATEDIFF(year, P.BIRTH_DT, AN.ACCT_ADMIT_DT) AS [Age at Admission],
    AN.ACCT_ADMIT_DT AS [Adm Date],
    AN.ACCT_DISCH_DT AS [Disch Date],
    T.CPT AS [CPT Code],
    PN.[Provider Description] AS [Visit Provider],
    PN.[EPIC_PROV_TYPE_NM] AS [Visit Provider Description],
    SUM(T.LineCharges) AS [Total Charges]
FROM dbo.Transactions AS T
    INNER JOIN dbo.Account_Number AS AN
        ON T.ACCOUNT_NUMBER = AN.ACCOUNT_NUMBER
    LEFT OUTER JOIN dbo.Provider AS PN
        ON AN.VST_PROV_NPI = PN.NPI
    INNER JOIN dbo.Patient AS P
        ON AN.MRN = P.MRN
GROUP BY 
    AN.ACCOUNT_NUMBER,
    T.Revcode,
    T.[RevCode Desc],
	P.BIRTH_DT,
    AN.ACCT_ADMIT_DT,
    AN.ACCT_DISCH_DT,
    T.CPT,
    PN.[Provider Description],
    PN.[EPIC_PROV_TYPE_NM];
GO

Commands completed successfully.

Total execution time: 00:00:00